In [16]:
%load_ext autoreload
%autoreload

import pyreadstat
import pandas as pd

from pa_lib.data import desc_col
from pa_lib.util import flatten
from pa_lib.file import store_excel, store_bin, store_pickle, write_xlsx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sbb, sbb_meta = pyreadstat.read_sav('C:/Users/kpf/data/sbbnutz_2014_1_5_final.sav')

In [3]:
def len_or_zero(obj):
    try:
        return len(obj)
    except TypeError:
        return 0

pd.DataFrame.from_records(
    data=[(e, 
           type(sbb_meta.__getattribute__(e)), 
           len_or_zero(sbb_meta.__getattribute__(e)), 
           sbb_meta.__getattribute__(e)) 
          for e in dir(sbb_meta) 
          if not str(e).startswith('__')], 
    columns=['name', 'type', 'len', 'obj'],
    index='name'
)

,type,len,obj
name,,,
column_labels,<class 'list'>,193,"[Interview ID, Address ID, Status, Fragebogenv..."
column_names,<class 'list'>,193,"[QID, QADRID, QSTATUS, QVERSION, QSTART, QEND,..."
file_encoding,<class 'str'>,12,WINDOWS-1252
file_format,<class 'str'>,8,sav/zsav
file_label,<class 'NoneType'>,0,None
missing_ranges,<class 'dict'>,0,{}
missing_user_values,<class 'list'>,0,[]
notes,<class 'list'>,0,[]
number_columns,<class 'int'>,0,193


In [4]:
val_lbl = sbb_meta.variable_value_labels
sbb_columns = pd.merge(
    left=pd.DataFrame(dict(col_name=sbb_meta.column_names, 
                           col_label=sbb_meta.column_labels,
                           col_orig_type=[dtype for (_, dtype) in sbb_meta.original_variable_types.items()]
                          )),
    right=pd.DataFrame(dict(col_name=list(val_lbl.keys()), 
                            codes=list(val_lbl.values())
                           )),
    how='left',
    on='col_name')

In [5]:
def dict_to_txt(d, sep=' | '):
    try:
        return sep.join([f'{k} => {v}' for (k, v) in d.items()])
    except AttributeError:
        return None

sbb_columns = sbb_columns.assign(code_txt=sbb_columns.codes.apply(dict_to_txt))

In [17]:
store_bin(sbb, 'sbb.feather')
store_pickle(sbb_columns, 'sbb_info.pkl')
write_xlsx(sbb, 'sbbnutz_data_2014.xlsx')
write_xlsx(sbb_columns, 'sbbnutz_columns_2014.xlsx')

2019-07-02 17:55:10 [INFO] Writing to file C:\Users\kpf\data\sbb.feather
2019-07-02 17:55:11 [INFO] Written 6.0 MB
2019-07-02 17:55:11 [INFO] Finished storing binary file in 0.07s (0.38s CPU)
2019-07-02 17:55:11 [INFO] Writing to file C:\Users\kpf\data\sbb_info.pkl
2019-07-02 17:55:11 [INFO] Written 70.5 KB
2019-07-02 17:55:11 [INFO] Writing to file C:\Users\kpf\data\sbbnutz_data_2014.xlsx
2019-07-02 17:55:17 [INFO] Written 1.6 MB
2019-07-02 17:55:17 [INFO] Finished writing xlsx file in 6.64s (7.23s CPU)
2019-07-02 17:55:17 [INFO] Writing to file C:\Users\kpf\data\sbbnutz_columns_2014.xlsx
2019-07-02 17:55:17 [INFO] Written 16.0 KB
2019-07-02 17:55:17 [INFO] Finished writing xlsx file in 0.05s (0.02s CPU)
